# Import Tiger and Regalloc

In [1]:
import tc

In [2]:
tc.has("regalloc")

True

In [3]:
import tiger_regalloc
tiger_regalloc == tc.regalloc

True

# Regalloc Library

In [4]:
list(filter(lambda e: not e.startswith("_") and not e.startswith("tiger"), dir(tc.regalloc)))

['allocate_registers']

# Tiger Magics and Regalloc

In [5]:
%%tiger ia32_executor ia32
let
    var b := 5
in
    print_int(b)
end

In [6]:
print(ia32_executor.result)
print(ia32_executor.tempmap)

5
fp -> ebp
rv -> eax
t0 -> eax
t3 -> ecx
t4 -> ecx
t5 -> ecx
t6 -> ebx
t7 -> edi
t8 -> esi
t9 -> ecx
t10 -> ecx
t11 -> ecx
t12 -> ecx
t13 -> ebx
t14 -> edi
t15 -> esi



In [7]:
%%tiger mips_executor mips
let
    var b := 5
in
    print_int(b)
end

In [8]:
print(mips_executor.result)
print(mips_executor.tempmap)

5
fp -> $fp
rv -> $v0
t46 -> $v0
t49 -> $a0
t50 -> $ra
t51 -> $ra
t52 -> $s0
t53 -> $s1
t54 -> $s2
t55 -> $s3
t56 -> $s4
t57 -> $s5
t58 -> $s6
t59 -> $s7
t61 -> $ra
t62 -> $ra
t63 -> $a0
t64 -> $s0
t65 -> $s1
t66 -> $s2
t67 -> $s3
t68 -> $s4
t69 -> $s5
t70 -> $s6
t71 -> $s7
t113 -> $ra
t114 -> $ra
t195 -> $ra
t196 -> $ra



# Regalloc example
In Tiger Magics it run regalloc and then run nolimips or gcc (see tcsh/python/ti.py)

In [9]:
import tempfile

with tempfile.NamedTemporaryFile() as f:
    f.write(b"let var b := 5 in print_int(b) end")
    f.seek(0)
    executor = tc.ti.TiExecutor(f.name)
    executor.init_target()
    data = executor.data
    data.lir_fragments = tc.target.lir_to_assem(data.fragments, data.target)

In [10]:
data.tempmap = data.target.cpu_get().tempmap_get()
data.tempmap = tc.regalloc.allocate_registers(data.lir_fragments, data.target, data.tempmap, tc.common.cvar.timer)
tc.target.frame_allocate(data.lir_fragments, data.target)

print(type(data.tempmap))
print(data.tempmap)

<class 'tiger_temp.TempMap'>
fp -> $fp
rv -> $v0
t237 -> $v0
t240 -> $a0
t241 -> $ra
t242 -> $ra
t243 -> $s0
t244 -> $s1
t245 -> $s2
t246 -> $s3
t247 -> $s4
t248 -> $s5
t249 -> $s6
t250 -> $s7
t252 -> $ra
t253 -> $ra
t254 -> $a0
t255 -> $s0
t256 -> $s1
t257 -> $s2
t258 -> $s3
t259 -> $s4
t260 -> $s5
t261 -> $s6
t262 -> $s7
t304 -> $ra
t305 -> $ra
t386 -> $ra
t387 -> $ra

